# Install and Import

In [1]:
!pip install Flask
!pip install firebase_admin

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from flask import Flask, request, jsonify
import time
from datetime import datetime, timezone
import io
import os
import firebase_admin # Import Firebase Admin SDK
from firebase_admin import credentials
from firebase_admin import firestore # Import Cloud Firestore



# Connect DataBase And APP

In [5]:


def initialize_firebase():
    global db
    try:
        # Kiểm tra xem app Firebase đã được khởi tạo chưa
        if not firebase_admin._apps:
            # Sử dụng thông tin xác thực từ file JSON
            cred = credentials.Certificate("/content/drive/MyDrive/Project/Dự án cntt 2 /dataset/density-traffic-app-firebase-adminsdk-fbsvc-0f73d667a9.json")
            # Khởi tạo Firebase Admin SDK
            firebase_admin.initialize_app(cred)
            print("Firebase Admin SDK initialized successfully.")

        # Lấy đối tượng Firestore client
        db = firestore.client()
        print("Firestore client obtained.")
        return True
    except FileNotFoundError:
        print(f"ERROR: Firebase service account key file not found at {cred}")
        return False
    except Exception as e:
        print(f"ERROR initializing Firebase: {e}")
        return False

# Config NGROK

In [6]:
!pip install pyngrok
!pip install flask-pyngrok

  Preparing metadata (setup.py) ... done
  Created wheel for flask-pyngrok: filename=Flask_PyNgrok-1.0.3-py3-none-any.whl size=2164 sha256=220653b55996358c0b0d332c5fd1d7536a6eb375e2ebb491ff4bf4d1a38e79a4
  Stored in directory: /root/.cache/pip/wheels/b2/02/ad/af386e8ac8b262575fb24016c3f2d21f5a96dade26cabb57c4
Successfully built flask-pyngrok


In [7]:
from pyngrok import ngrok
import threading
import time
import logging



In [8]:
!ngrok authtoken 2wxoCJhRUriCw9aQDK5Vcbab88R_4gnHxxQ1CdaXQFScr8CVs

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
ngrok.kill()


In [10]:
app = Flask(__name__)
initialize_firebase()

Firebase Admin SDK initialized successfully.
Firestore client obtained.


True

In [11]:
@app.route('/api/traffic/latest_densities', methods=['GET'])
def get_latest_densities_endpoint():
    """
    Endpoint to fetch the latest traffic density data from Firestore for requested cameras.
    Receives camera_ids (string) via query parameter.
    Returns a list of TrafficDensityReading objects (excluding the 'maximum' field).
    """
    # Check if the Firestore client is available
    if db is None:
        logging.error("Firestore client is not available in get_latest_densities_endpoint.")
        return jsonify({"error": "Database not initialized"}), 500

    camera_ids_param = request.args.get('camera_ids')

    if not camera_ids_param:
        logging.warning("No 'camera_ids' query parameter provided for GET request.")
        return jsonify([]), 200 # Return an empty list as it's safe and valid


    requested_camera_ids = [id.strip() for id in camera_ids_param.split(',') if id.strip()]

    if not requested_camera_ids:
        logging.warning("Provided 'camera_ids' parameter is empty or contains only whitespace after splitting.")
        return jsonify([]), 200 # Return an empty list if no valid IDs after splitting

    logging.info(f"Received GET request for camera IDs: {requested_camera_ids}")

    results = [] # List to store results for the cameras

    # Loop through each requested camera ID
    for camera_id in requested_camera_ids:
        try:
            # Reference the camera's document in the 'cameras' collection
            # Assuming each camera has a document with its ID as the document ID
            camera_ref = db.collection('cameras').document(camera_id)

            # Get the current document
            doc = camera_ref.get()

            # Check if the document exists
            if doc.exists:
                doc_data = doc.to_dict()
                # logging.info(f"Found data for camera ID {camera_id}: {doc_data}") # This log might be too verbose

                # Extract necessary data
                # Use .get() with default values to avoid KeyError if fields are missing
                density = doc_data.get('density', 0.0) # Density (0.0-1.0 ratio)
                timestamp = doc_data.get('timestamp', 0) # Timestamp (milliseconds)
                # maximum = doc_data.get('maximum', 0.0) # Exclude the 'maximum' field as requested
                summary = doc_data.get('summary', '') # Summary text

                # Create the result dictionary matching the Android TrafficDensityReading class
                # Ensure key names match the @SerializedName in the Android data class
                result_item = {
                    "camera_id": camera_id,
                    "timestamp": timestamp,
                    "density": density,
                    # "maximum": maximum, # Exclude the maximum field from the GET response
                    "summary": summary # Include the summary text in the response
                }
                results.append(result_item)

            else:
                logging.warning(f"No data found in Firestore for camera ID: {camera_id}")
                # Add an item to the results indicating that no data was found for this ID
                results.append({
                    "camera_id": camera_id,
                    "timestamp": 0, # Or current timestamp if preferred
                    "density": 0.0,
                    # "maximum": 0.0, # Exclude the maximum field
                    "summary": "No data found",
                    "status": "not_found" # Add a status field for the client
                })

        except Exception as e:
            # Handle errors that occur while fetching data from Firestore
            logging.error(f"Error fetching data for camera ID {camera_id} from Firestore: {e}")
            # Add an item to the results indicating an error for this ID
            results.append({
                 "camera_id": camera_id,
                 "timestamp": 0,
                 "density": 0.0,
                 # "maximum": 0.0, # Exclude the maximum field
                 "summary": f"Error fetching: {e}",
                 "status": "error" # Add a status field for the client
            })


    # Return the list of results as JSON
    # jsonify will automatically set the Content-Type to application/json
    return jsonify(results), 200 # Return the list of results and status code 200 OK


In [12]:
@app.route('/')
def home():
    """Root endpoint, returns a welcome message or status."""
    logging.info("Accessed root endpoint.")
    return "Traffic Density API Backend is running!"

In [13]:
!lsof -i :5000

In [ ]:
#!kill 349

In [ ]:
if __name__ == '__main__':
    initialize_firebase()

    if firebase_admin._apps and db is not None:
        logging.info("Starting Flask server and Ngrok tunnel...")

        FLASK_PORT = 5000

        tunnel = None
        public_url = None

        try:

            ngrok.kill()
            logging.info("Killed existing ngrok tunnels.")

            time.sleep(2)
            logging.info("Waited 2 seconds after killing ngrok.")

            tunnel = ngrok.connect(FLASK_PORT)
            public_url = tunnel.public_url

            logging.info(f"Ngrok tunnel created: {public_url}")
            print(f" * Ngrok URL: {public_url}")

            logging.info("Starting Flask app in a separate thread...")
            threading.Thread(target=app.run, kwargs={'host': '0.0.0.0', 'port': FLASK_PORT, 'debug': False, 'use_reloader': False}).start()
            logging.info(f"Flask app thread started on port {FLASK_PORT}.")

            print("Press Ctrl+C in this cell to stop the tunnel and application.")
            try:

                stop_event = threading.Event()
                stop_event.wait()
            except KeyboardInterrupt:
                logging.info("\nInterrupt received, stopping application...")
            finally:
                if tunnel:
                    ngrok.disconnect(public_url)
                    logging.info("Ngrok tunnel disconnected.")
                logging.info("Application stopped.")

        except Exception as e:
            logging.error(f"An error occurred while setting up or running Flask/Ngrok: {e}")
            import traceback
            logging.error(traceback.format_exc())
            print(f" * Error: {e}")

    else:
        logging.error("Flask server cannot start because Firebase was not initialized successfully.")


Firestore client obtained.
 * Ngrok URL: https://eadf-35-247-40-140.ngrok-free.app
Press Ctrl+C in this cell to stop the tunnel and application.
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [13/May/2025 15:08:26] "GET /api/traffic/latest_densities?camera_ids=66f126e8538c780017c9362f HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 15:08:39] "GET /api/traffic/latest_densities?camera_ids=66f126e8538c780017c9362f HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 15:08:47] "GET /api/traffic/latest_densities?camera_ids=662b4e8e1afb9c00172d865c HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 15:09:01] "GET /api/traffic/latest_densities?camera_ids=662b4e8e1afb9c00172d865c HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [13/May/2025 15:09:16] "GET /api/traffic/latest_densities?camera_ids=662b4e8e1afb9c00172d865c HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [1